In [1]:
import os
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np
from ResNet import ResidualUnit
from Inception import InceptionUnit

In [2]:
model_resnet34 = keras.models.Sequential()
model_resnet34.add(keras.layers.Conv2D(64, 7, strides = 2, input_shape = [224, 224, 3], padding = "same", use_bias = False))
model_resnet34.add(keras.layers.BatchNormalization())
model_resnet34.add(keras.layers.Activation("relu"))
model_resnet34.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding = "same"))
prev_filters = 64
for filters in [64, 64, 128, 128, 256, 256, 512, 512]:
    strides = 1 if filters == prev_filters else 2
    model_resnet34.add(ResidualUnit(filters, strides = strides))
    prev_filters = filters
model_resnet34.add(keras.layers.GlobalAveragePooling2D())
model_resnet34.add(keras.layers.Flatten())
model_resnet34.add(keras.layers.Dense(10, activation = "softmax"))

In [20]:
model_googlenet = keras.models.Sequential()
model_googlenet.add(keras.layers.Conv2D(64, 7, strides = 2, input_shape = [224, 224, 3], padding = "same", use_bias = False))
model_googlenet.add(keras.layers.BatchNormalization())
model_googlenet.add(keras.layers.Activation("relu"))
model_googlenet.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding = "same"))
model_googlenet.add(InceptionUnit())
model_googlenet.add(keras.layers.GlobalAveragePooling2D())
model_googlenet.add(keras.layers.Flatten())
model_googlenet.add(keras.layers.Dense(1, activation = "sigmoid"))

In [ ]:
img_height = 224
img_weight = 224
batch_size = 16

In [ ]:
ds_train = keras.utils.image_dataset_from_directory(
    ""E:\Datasets"",
    labels = 'inferred',
    label_mode = "int",
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (img_height, img_weight),
    shuffle = True,
    seed = 123,
    validation_split = 0.20,
    subset = "training"
)

In [ ]:
ds_valid = keras.utils.image_dataset_from_directory(
    ""E:\Datasets"",
    labels = 'inferred',
    label_mode = "int",
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (img_height, img_weight),
    shuffle = True,
    seed = 123,
    validation_split = 0.20,
    subset = "validation"
)

In [ ]:
NAME = "tender-cnn1-{}".format(int(time.time()))
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/{}".format(NAME))

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model_googlenet.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history = model_googlenet.fit(ds_train,
                    epochs=1,
                    validation_data=ds_valid,
                    callbacks=[tensorboard]
                   )